In [1]:
# 메모리내 변수 제거

all = [var for var in globals() if var[0] != "_"]   # globals() 목록의 첫글자가 _ 로 시작하지 않는 자료의 리스트만 가져와서
for var in all:
    del globals()[var]

In [2]:
#%pip install xgboost
#%pip install wordcloud
#%pip install gensim

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import accuracy_score
## for data
import pandas as pd

import re
import nltk## for language detection

# 박스 출력
import textwrap

# 데이터 수집

In [4]:
train = pd.read_csv('c:/data/project/train.csv')
test = pd.read_csv('c:/data/project/test.csv')

train.head()

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1


In [5]:
train['first_party_winner'].value_counts()

first_party_winner
1    1649
0     829
Name: count, dtype: int64

# 데이터 전처리

In [6]:
df = train[['facts', 'first_party_winner']]
df_target = df[['first_party_winner']]
df_nlp = df[['facts']]
df_nlp1 = pd.DataFrame(df_nlp, columns=['facts'])
df_nlp1['facts'] = df_nlp1['facts'].str.replace(r'<[^<>]*>', '', regex=True) # 특수 문자 제거
df_nlp1['facts'] = df_nlp1['facts'].str.replace(r'\d', '', regex=True)  # 숫자 제거

In [7]:
corpus = df_nlp1['facts']
# print(corpus.str.cat(sep=" ")) # 인덱스의 요소들 서로 잇기
lst_tokens = nltk.tokenize.word_tokenize(corpus.str.cat(sep=" "))
ps = nltk.stem.porter.PorterStemmer()
lem = nltk.stem.wordnet.WordNetLemmatizer()
lst_stopwords = nltk.corpus.stopwords.words("english")

In [8]:
# 문장 깨끗하게 만드는 함수

def utils_preprocess_text(text, flg_stemm=True, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    import re
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    
    ## Tokenize (convert from string to list)
    lst_text = text.split()    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list # 역토큰화, 벡터화 하기 위해서
    text = " ".join(lst_text)
    return text

In [9]:
# 문장 깨끗하게 만드는 함수 사용
df_nlp1["facts_clean"] = df_nlp1["facts"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords))
corpus_stopwords = df_nlp1["facts_clean"]
lst_tokens_stopwords = nltk.tokenize.word_tokenize(corpus_stopwords.str.cat(sep=" "))
stop_words=[]
# 갯수가 하나만 있는 단어들 제거
for word, freq in nltk.FreqDist(lst_tokens_stopwords).most_common():
    if freq == 1:
        #print(word)
        stop_words.append(word)

df_nlp1["facts_clean"] = df_nlp1["facts_clean"] .apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=False, lst_stopwords=stop_words))
# y값 포함해서 하나의 프레임으로 만들기


In [10]:
# 모델링 전 완전 데이터
df_nlp2 = pd.concat([df_nlp1,df_target['first_party_winner']],axis=1, join='inner')
df_nlp2 = df_nlp2.drop(columns='facts')
df_nlp2

,facts_clean,first_party_winner
0,june phil st amant candidate public office mad...,1
1,ramon nelson riding bike suffered lethal blow ...,0
2,alabama state court convicted billy joe magwoo...,1
3,victor linkletter convicted state court eviden...,0
4,april selma alabama intruder broke apartment d...,1
...,...,...
2473,congress amended clean air act energy policy a...,1
2474,alliance bond fund inc investment fund purchas...,1
2475,district court sentenced manuel peguero month ...,0
2476,march st cyr lawful permanent resident pled gu...,0


In [11]:
#TFIDF / 전체 문서들 중에서 빈도수가 높으면 가중치가 낮고, 특정 문서에서 빈도수가 높으면 가중치를 높게 줌

# vectorizeTF = TfidfVectorizer()
# count_matrix_tf = vectorizeTF.fit_transform(df_nlp2['facts_clean'])
# data_final = count_matrix_tf.toarray()
# data_final = pd.DataFrame(data=data_final, columns=vectorizeTF.get_feature_names_out())
# data_final = pd.concat([data_final,df_nlp2["first_party_winner"]],axis=1,join='inner')
# terms = vectorizeTF.get_feature_names_out()
# data_final

In [12]:
# 카운터 벡터라이즈 / 전체 문서에서 빈도수가 높으면 가중치가 높음
   
# vectorizeCV=CountVectorizer()
# count_matrix = vectorizeCV.fit_transform(df_nlp2['facts_clean'])
# count_array = count_matrix.toarray()
# data_final = pd.DataFrame(data=count_array,columns = vectorizeCV.get_feature_names_out())
# data_final = pd.concat([data_final,df_nlp2["first_party_winner"]],axis=1,join='inner')
# terms = vectorizeCV.get_feature_names_out()
# data_final

In [13]:
def chooseVec(vec,nlp,testSet=False):
    """
        벡터 고르는 함수입니다.
        vec 파라미터로 tf-idf이면 'tf', count-vector는 'cv'.
        nlp는 깨끗하게 전처리된 데이터셋을 보내주세요.
    """
    global terms, vectorize

    if testSet == True: # 테스트 셋일 경우, 새로운 인스턴스를 부여하면 안되니까
        count_matrix = vectorize.transform(nlp['facts_clean'])
    
    if vec == 'tf' and testSet == False:
        #TFIDF / 전체 문서들 중에서 빈도수가 높으면 가중치가 낮고, 특정 문서에서 빈도수가 높으면 가중치를 높게 줌
        vectorize = TfidfVectorizer()
        count_matrix = vectorize.fit_transform(nlp['facts_clean'])

    if vec == 'cv'and testSet == False:
        # 카운터 벡터라이즈 / 전체 문서에서 빈도수가 높으면 가중치가 높음
        vectorize=CountVectorizer()
        count_matrix = vectorize.fit_transform(nlp['facts_clean'])

    count_array = count_matrix.toarray()
    data_final = pd.DataFrame(data=count_array,columns = vectorize.get_feature_names_out())
    data_final = pd.concat([data_final,nlp["first_party_winner"]],axis=1,join='inner')
    terms = vectorize.get_feature_names_out()
    testSet = False
    return data_final
    

In [14]:
data_final = chooseVec('tf', df_nlp2) # td-idf : 'tf' / counter vector : 'cv'

# 데이터 모델링

In [15]:
# X_train, X_test, y_train, y_test = train_test_split(data_final.drop(columns=['first_party_winner']), data_final['first_party_winner'], test_size=0.3,random_state=0)

In [16]:
# from sklearn.decomposition import LatentDirichletAllocation
# lda = LatentDirichletAllocation(n_components=200, random_state=1) # n_component : 토픽 갯수
# pd.DataFrame(lda.fit_transform(X_train))

In [17]:
# 데이터 쪼개기
def splitData(data):
    global X_train, X_test, y_train, y_test
    X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['first_party_winner']), data['first_party_winner'], test_size=0.3,random_state=0)

# LDA 학습
# def ldaTraining(data, n):
#     """
#         X_trian과 lDA 적용 후 확인할 데이터 n개
#     """
#     # LDA 학습, 단어의 의미구조 파악
#     # 30초정도 걸림
#     global lda, X_train
#     from sklearn.decomposition import LatentDirichletAllocation
#     lda = LatentDirichletAllocation(n_components=200, random_state=1) # n_component : 토픽 갯수
#     lda_data = lda.fit_transform(data)
#     X_train = pd.DataFrame(data=lda_data)
#     # LDA 학습 결과 보기
#     for idx, topic in enumerate(lda.components_):
#         print("Topic %d:" % (idx+1), [(terms[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])


# 모델 만들기
def modeling(params):
    """
        XGB 학습 모델 함수.
        XGB 파라미터를 받습니다.
    """
    global model
    model = XGBClassifier(**params) # **param_dict
    model.fit(X_train, y_train)
    print(model.get_params())

# 예측하기
def prediction(data):
    """
        테스트 데이터 예측하기
    """
    # lda_data_t = pd.DataFrame(data=lda.transform(data)) # 테스트 lda 적용
    y_pred = model.predict(data)
    y_pred_df = pd.DataFrame(data=y_pred, columns=['data'])
    display(y_pred_df.value_counts())

    # 평가
    from sklearn.metrics import f1_score
    accuracy = accuracy_score(y_test, y_pred) # 정확도
    print("Accuracy : %.2f%%" % (accuracy * 100.0)) 
    print('f1_score :', f1_score(y_test, y_pred)) # 정밀도와 재현율의 조화평균
    print("model score (X_train, y_train):", model.score(X_test, y_test))

In [18]:
def fullTraining(data):
    # 훈련
    splitData(data)
    # ldaTraining(X_train, 5)

    # XG부스트 모델 적용하기
    default_gs_params = {'learning_rate': 0.6, 'max_depth': 9, 'scale_pos_weight': 0.3, 'subsample': 0.9}
    modeling(default_gs_params)

fullTraining(data_final)

{'objective': 'binary:logistic', 'use_label_encoder': None, 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'gpu_id': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.6, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 9, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'n_estimators': 100, 'n_jobs': None, 'num_parallel_tree': None, 'predictor': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': 0.3, 'subsample': 0.9, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}


In [19]:
# 테스트 적용
prediction(X_test)

data
1       516
0       228
Name: count, dtype: int64

Accuracy : 58.20%
f1_score : 0.6886886886886887
model core (X_train, y_train): 0.581989247311828


In [20]:
# 그리드서치로 최적의 파라미터 값 찾아보기

def findParam():
    from sklearn.model_selection import GridSearchCV
    param_xgb = {"scale_pos_weight": [0.3],
                 "learning_rate":[0.3, 0.6, 0.9], # 0~1 높을 수록 과적합 된다, 만약에 값이 낮으면 n_estimators를 높여야 과적합이 방지
                 "max_depth":[3,5,7,9], # 보통 3~10, 높을 수록 과적합
                 "subsample":[0.5, 0.7, 0.9] # 학습에 사용하는 샘플링 비율 0.5 ~ 1, 높을 수록 과적합
                }    

    gscv_xgb = GridSearchCV (estimator = model, param_grid = param_xgb, scoring ='f1', cv = 3, refit=True, n_jobs=1, verbose=2)
    gscv_xgb.fit(X_train, y_train)
    gs_params= gscv_xgb.best_params_
    print('XGB 파라미터: ', gs_params)
    print('XGB 예측 정확도: {:.4f}'.format(gscv_xgb.best_score_))

    return gs_params
# 그리드서치 후 파라미터 적용해서 모델링 학습 다시 하기
# modeling(findParam())


### 테스트csv 적용

In [21]:
dfTest = pd.DataFrame(test['facts'])
dfTest['facts'] = dfTest['facts'].str.replace(r'<[^<>]*>', '', regex=True)
dfTest['facts'] = dfTest['facts'].str.replace(r'\d', '', regex=True)  # 숫자 제거
dfTest["facts_clean"] = dfTest["facts"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords))
dfTest['first_party_winner'] = np.zeros(len(dfTest)).astype(int)
dfTest = dfTest.drop(columns='facts')


In [22]:
dfTest

,facts_clean,first_party_winner
0,bail reform act allowed federal court detain a...,0
1,lexecon inc defendant class action lawsuit usc...,0
2,fox television station broadcast billboard mus...,0
3,trial armed robbery federally insured saving l...,0
4,magistrate judge issued warrant authorizing se...,0
...,...,...
1235,according executive order signed president geo...,0
1236,section clean air act caa requires environment...,0
1237,linda matteo john madigan created plan utilizi...,0
1238,north carolina board agriculture adopted regul...,0


In [23]:
# 벡터 고르기
data_final_test = chooseVec('tf', dfTest, True)
data_final_test

,__,aa,aacw,aai,aar,aaron,ab,abandoned,abandonment,abatement,...,zj,zone,zoned,zoneofinterests,zoning,zuni,zurich,zurko,zurkos,first_party_winner
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1236,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [24]:
data_final_test

,__,aa,aacw,aai,aar,aaron,ab,abandoned,abandonment,abatement,...,zj,zone,zoned,zoneofinterests,zoning,zuni,zurich,zurko,zurkos,first_party_winner
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1236,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [25]:
# splitData(data_final_test)
def testSplit(data):
    global X_test, y_test
    X_test = data.drop(columns=['first_party_winner'])
    y_test = data['first_party_winner']
testSplit(data_final_test)

In [26]:
def testPredict():
    global predcsv, X_test
    
    # X_test = dataTest.drop(columns=['first_party_winner'])
    # y_test = dataTest['first_party_winner']
    # X_test = pd.DataFrame(data=lda.transform(X_test))
    y_pred_test = model.predict(X_test)
    predcsv = pd.DataFrame(y_pred_test, columns=['first_party_winner'])

testPredict()

In [27]:
def submission():
    submit = pd.read_csv('C:/data/project/sample_submission.csv')
    submit['first_party_winner'] = predcsv
    submit.to_csv('./sample_submission.csv', index=False)
    print('Done')

In [28]:
predcsv.value_counts()

first_party_winner
1                     878
0                     362
Name: count, dtype: int64

In [29]:
submission()

Done


### 셀프 트레이닝

def findGoodSample():
    # 데이터 준비
    proba = model.predict_proba(X_test)
    proba_df = pd.DataFrame(data=proba, columns=['proba_0','proba_1'])

    # 0보다 1의 확률이 높은 데이터 추출
    proba_higher = proba_df[proba_df['proba_1'] > proba_df['proba_0']]
    # 1이 될 확률이 90프로 이상인 자료의 인덱스
    proba_higher_index = proba_higher[proba_higher['proba_1'] > 0.9].index.to_list()

    print('추가되는 샘플 갯수 :',len(proba_higher_index))
    display(proba_higher_index)
    # 인덱스 번호 리턴
    return proba_higher_index


dfTest

def concatSample():
    newIdx = findGoodSample()
    filtered = dfTest.loc[dfTest.index.isin(newIdx)].drop(columns='facts')
    filtered['first_party_winner'] = 1
    dataTest = pd.concat([df_nlp2,filtered]).reset_index(drop=True)
    print('기존 샘플 갯수 :', len(df_nlp2))
    print('새로운 프레임 데이터 갯수 :', len(dataTest))
    return dataTest

data_final_test = chooseVec('tf', concatSample(), True)
fullTraining(data_final_test)
prediction(X_test)

# testPredict(X_test)

# predcsv.value_counts()